In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)

# HOLC grades to 2020 tracts

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

tract_2020 = gpd.read_file(r'Data\US_tract_2020.shp')
holc = gpd.read_file(r'Data\holc_ad_data.shp')
tract_2020 = tract_2020.to_crs("ESRI:102003")
holc = holc.to_crs("ESRI:102003")

tract_2020['tract_area_m2'] = tract_2020['geometry'].area
tract_holc = gpd.overlay(tract_2020, holc, how='intersection')

# Calculate area and area percentage
tract_holc['intersect_area_m2'] = tract_holc['geometry'].area
tract_holc['area_pct'] = tract_holc['intersect_area_m2'] / tract_holc['tract_area_m2']

# Summarize data to determine overlay percentage by tract
area_summary = tract_holc.groupby(['GEOID'])['area_pct'].sum().reset_index() 

# Merge this summary back to the original tract dataframe
tract_2020 = tract_2020.merge(area_summary, on='GEOID', how='left')


### Assign categories based on the overlay percentage
# threshold = 0.25

# Fill NA values with 0 for tracts with no overlays
tract_2020['area_pct'].fillna(0, inplace=True)

def categorize_overlay(area_pct):
    if area_pct == 0:
        return 'no_overlay'
    elif 0 < area_pct < 0.25:
        return 'less_than_threshold'
    elif area_pct >= 0.25:
        return 'greater_or_equal_threshold'

tract_2020['overlay_category'] = tract_2020['area_pct'].apply(categorize_overlay)
tract_2020.groupby('overlay_category')['area_pct'].describe()

,count,mean,std,min,25%,50%,75%,max
overlay_category,,,,,,,,
greater_or_equal_threshold,12853.0,0.780683,0.238190,2.501588e-01,0.588034,0.870852,1.00000,1.907672
less_than_threshold,3746.0,0.080109,0.079092,5.500677e-08,0.006929,0.053454,0.14471,0.249897
no_overlay,67815.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000


In [3]:
# For tracts above the threshold, calculate the dominant HOLC grade
# Group by GEOID and HOLC grade to sum areas
area_by_grade = tract_holc.groupby(['GEOID', 'holc_grade'])['area_pct'].sum().reset_index()

# Find the index of the maximum percentage
idx = area_by_grade.groupby('GEOID')['area_pct'].idxmax()
max_area_grade = area_by_grade.loc[idx][['GEOID', 'holc_grade']]

# Merge the maximum HOLC grade back to the main DataFrame 
tract_2020 = tract_2020.merge(max_area_grade, on='GEOID', how='left')

tract_2020.head(2)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract_area_m2,area_pct,overlay_category,holc_grade
0,01,045,020801,01045020801,208.01,Census Tract 208.01,G5020,S,19261326.0,0.0,+31.3971437,-085.6615959,"POLYGON ((974088.803 -630642.090, 974105.477 -...",1.926132e+07,0.0,no_overlay,NaN
1,01,017,954200,01017954200,9542,Census Tract 9542,G5020,S,62998696.0,34641.0,+32.8341436,-085.2678516,"POLYGON ((992602.687 -469304.620, 992744.903 -...",6.303334e+07,0.0,no_overlay,NaN


In [4]:
# Describe those above 0.25 threshold
tract_2020[tract_2020.overlay_category == 'greater_or_equal_threshold'].groupby('holc_grade').count()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract_area_m2,area_pct,overlay_category
holc_grade,,,,,,,,,,,,,,,,
A,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795
B,2568,2568,2568,2568,2568,2568,2568,2568,2568,2568,2568,2568,2568,2568,2568,2568
C,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873,5873
D,3609,3609,3609,3609,3609,3609,3609,3609,3609,3609,3609,3609,3609,3609,3609,3609
E,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8


# 1940 tracts and sociodemographics

In [5]:
# Read US Tract in 1940
tract_1940 = gpd.read_file(r'Data\US_tract_1940.shp')

# length of tract_1940
print(tract_1940.shape[0])

# CRS of tract_1940
print(tract_1940.crs) # esri 102003

# Calculate area for each tract
tract_1940['area_km2_1940_tract'] = tract_1940.geometry.area / 1000000

# First 2 rows
tract_1940.head(2)

7563
PROJCS["USA_Contiguous_Albers_Equal_Area_Conic",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",37.5],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102003"]]


,NHGISST,NHGISCTY,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,geometry,area_km2_1940_tract
0,550,0790,G55007900090,55007900090,1.902826e+06,9640.965368,"POLYGON ((650791.784 654746.221, 650886.198 65...",1.902826
1,550,0790,G55007900094,55007900094,4.469542e+06,12723.051874,"POLYGON ((646759.565 651779.856, 646851.837 65...",4.469542


In [6]:
# Read the 1940 sociodemographic data & clean it

info = pd.read_csv(r'Data\nhgis_1940_tract.csv')
print(info.shape[0])
info.head(2)

# no black & other race info; use non white
info = info.iloc[1:]

for col in info.columns[10:]: 
    info[col] = pd.to_numeric(info[col], errors='coerce')
    
info['pct_black'] = 100 * info['BVG001'] / info['BUB001'] 
info['pct_nonwhite'] = 100 * info['BUQ002'] / info['BUB001']
info['pct_foreign_white'] = 100 * (info['BVT003'] + info['BVT004']) / info['BUQ001']
info['pct_college_above'] = 100 * (info['BUH007'] + info['BUH008'] + info['BUH016'] + info['BUH017']) / info['BUB001']

# unempl: 
# 1 - (Male In labor force: Employed  + Female In labor force: Employed )/ (male in labor force + female in labor force)
info['pct_unempl'] = 100 * (1 - ((info['BUX001'] + info['BUX009']) / (info['BUW001'] + info['BUW003'])))

info['pct_renter']= 100 * info['BU2002'] / (info['BU2001'] + info['BU2002'])

info = info.rename(columns = {'BUB001':'total_pop', 'BVC001':'median_h_value'})

info['median_h_value'] = info['median_h_value'] / 1000

info = info[['GISJOIN', 'total_pop', 'pct_black', 'pct_nonwhite', 'pct_foreign_white', 
             'median_h_value', 'pct_college_above', 
             'pct_unempl', 'pct_renter']]

info['GISJOIN'] = info['GISJOIN'].str.zfill(12)

info.head()

7468


,GISJOIN,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter
1,G01007300001,2792,0.322350,0.322350,0.431189,2.435,5.336676,13.008937,50.573066
2,G01007300002,1796,1.614699,1.614699,1.358234,2.444,4.398664,16.268657,58.298755
3,G01007300003,3935,4.040661,4.040661,0.582627,2.685,6.429479,12.204234,62.118321
4,G01007300004,4882,15.116755,15.116755,0.579151,2.284,2.929127,14.592275,51.386623
5,G01007300005,5960,27.449664,27.449664,0.994450,1.842,2.953020,18.115002,60.804672


In [7]:
# Some further cleaning

# Median household value
print('0 home value--removed', info[info.median_h_value == 0].shape[0])
info = info[info.median_h_value != 0]
print(info.shape[0])

# Pct foreign white
print(info.isna().sum())
# remove nan values for pct_foreign_white--2
info = info[info.pct_foreign_white.notna()]
print(info.shape[0])

0 home value--removed 1408
6059
GISJOIN              0
total_pop            0
pct_black            0
pct_nonwhite         0
pct_foreign_white    2
median_h_value       0
pct_college_above    0
pct_unempl           0
pct_renter           0
dtype: int64
6057


In [8]:
# Join 1940 boundaries with 1940 sociodemographics

tract_1940 = pd.merge(tract_1940, info, how = 'left', on = 'GISJOIN')
print(tract_1940[tract_1940.total_pop.isna()].shape[0])
#info2 = info2[info2.total_pop.notnull()]
print(tract_1940.shape[0])

1506
7563


# Join 1940 sociodemograpics and 2020 data

In [9]:
# Spatial join
# If the centroid of an 2020 tract is in 1940 tract, the 2020 tract uses the same socioeconomic info of 1940 tract

# Create centroid of tract_2020
tract_2020['centroid'] = tract_2020.geometry.centroid
centroid_2020 = tract_2020.set_geometry('centroid')

# Spatial join - join info from tract_1940 to centroid of tract_2020
joined = gpd.sjoin(centroid_2020, tract_1940, op='within', how='left')
print(joined.shape[0])
joined.head(2)

C:\Users\snow_\anaconda3\envs\general_env\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


84414


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract_area_m2,area_pct,overlay_category,holc_grade,centroid,index_right,NHGISST,NHGISCTY,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,area_km2_1940_tract,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter
0,01,045,020801,01045020801,208.01,Census Tract 208.01,G5020,S,19261326.0,0.0,+31.3971437,-085.6615959,"POLYGON ((974088.803 -630642.090, 974105.477 -...",1.926132e+07,0.0,no_overlay,NaN,POINT (977621.078 -628908.123),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,017,954200,01017954200,9542,Census Tract 9542,G5020,S,62998696.0,34641.0,+32.8341436,-085.2678516,"POLYGON ((992602.687 -469304.620, 992744.903 -...",6.303334e+07,0.0,no_overlay,NaN,POINT (996560.912 -464710.571),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print(joined[(joined['total_pop'].notna())&(joined.overlay_category == 'greater_or_equal_threshold')].shape[0])

joined['overlap20_40'] = ((joined['total_pop'].notna()) & (joined.overlay_category == 'greater_or_equal_threshold')).astype(int)

joined.head(2)

7971


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract_area_m2,area_pct,overlay_category,holc_grade,centroid,index_right,NHGISST,NHGISCTY,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,area_km2_1940_tract,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter,overlap20_40
0,01,045,020801,01045020801,208.01,Census Tract 208.01,G5020,S,19261326.0,0.0,+31.3971437,-085.6615959,"POLYGON ((974088.803 -630642.090, 974105.477 -...",1.926132e+07,0.0,no_overlay,NaN,POINT (977621.078 -628908.123),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,01,017,954200,01017954200,9542,Census Tract 9542,G5020,S,62998696.0,34641.0,+32.8341436,-085.2678516,"POLYGON ((992602.687 -469304.620, 992744.903 -...",6.303334e+07,0.0,no_overlay,NaN,POINT (996560.912 -464710.571),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# Join health data

In [11]:
# Read CDC PLACES data

cdc = pd.read_csv('Data\PLACES_2022_release.csv')
print(cdc.shape[0])
cdc['TractFIPS'] = cdc['TractFIPS'].astype(str).str.zfill(11)
# cdc[cdc['OBESITY_CrudePrev'].isna()] # no nan values
cdc = cdc[['TractFIPS', 'StateAbbr', 'OBESITY_CrudePrev', 'DIABETES_CrudePrev', 'BPHIGH_CrudePrev']]
print(cdc[cdc['BPHIGH_CrudePrev'].isna()].shape[0]) # 1999 tracts lack hypertension info but not obesity and diabetes.
cdc.rename(columns = {'OBESITY_CrudePrev':'obesity', 
                           'DIABETES_CrudePrev':'diabetes', 
                           'BPHIGH_CrudePrev':'bphigh'}, inplace = True)
 
cdc.head(2)

72337
1999


,TractFIPS,StateAbbr,obesity,diabetes,bphigh
0,37051003402,NC,24.2,1.7,11.2
1,04013422643,AZ,28.3,6.4,23.7


In [12]:
# join the data

df_join3 = pd.merge(joined, cdc, left_on = 'GEOID', right_on = 'TractFIPS', how = 'left')
df_join3.head(2)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,tract_area_m2,area_pct,overlay_category,holc_grade,centroid,index_right,NHGISST,NHGISCTY,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,area_km2_1940_tract,total_pop,pct_black,pct_nonwhite,pct_foreign_white,median_h_value,pct_college_above,pct_unempl,pct_renter,overlap20_40,TractFIPS,StateAbbr,obesity,diabetes,bphigh
0,01,045,020801,01045020801,208.01,Census Tract 208.01,G5020,S,19261326.0,0.0,+31.3971437,-085.6615959,"POLYGON ((974088.803 -630642.090, 974105.477 -...",1.926132e+07,0.0,no_overlay,NaN,POINT (977621.078 -628908.123),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,01045020801,AL,41.5,15.0,44.5
1,01,017,954200,01017954200,9542,Census Tract 9542,G5020,S,62998696.0,34641.0,+32.8341436,-085.2678516,"POLYGON ((992602.687 -469304.620, 992744.903 -...",6.303334e+07,0.0,no_overlay,NaN,POINT (996560.912 -464710.571),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,01017954200,AL,39.8,13.7,44.4


In [13]:
# Assign states
region_dict = {
    'CT': 'Northeast', 'ME': 'Northeast', 'MA': 'Northeast', 'NH': 'Northeast', 'NJ': 'Northeast', 'NY': 'Northeast', 'PA': 'Northeast', 'RI': 'Northeast', 'VT': 'Northeast',
    'IL': 'Midwest', 'IN': 'Midwest', 'IA': 'Midwest', 'KS': 'Midwest', 'MI': 'Midwest', 'MN': 'Midwest', 'MO': 'Midwest', 'NE': 'Midwest', 'ND': 'Midwest', 'OH': 'Midwest', 'SD': 'Midwest', 'WI': 'Midwest',
    'AL': 'South', 'AR': 'South', 'DE': 'South', 'FL': 'South', 'GA': 'South', 'KY': 'South', 'LA': 'South', 'MD': 'South', 'MS': 'South', 'NC': 'South', 'OK': 'South', 'SC': 'South', 'TN': 'South', 'TX': 'South', 'VA': 'South', 'WV': 'South',
    'AK': 'West', 'AZ': 'West', 'CA': 'West', 'CO': 'West', 'HI': 'West', 'ID': 'West', 'MT': 'West', 'NV': 'West', 'NM': 'West', 'OR': 'West', 'UT': 'West', 'WA': 'West', 'WY': 'West'
}

# Map the state abbreviations to the regions
df_join3['Region'] = df_join3['StateAbbr'].map(region_dict)

dummies = pd.get_dummies(df_join3['Region'], prefix='Region').astype(int) # add astype(int)
df_join3 = pd.concat([df_join3, dummies], axis=1)

In [14]:
# Sample size for above 0.25 threshold without missing data for obesity

df_join3[(df_join3.overlap20_40 == 1)&
         (df_join3.holc_grade.isin(['A','B','C','D']))&
         (df_join3.obesity.notna())
        ].shape

(6981, 45)

In [15]:
# Describe the sample

df_join3[(df_join3.overlap20_40 == 1)&
         (df_join3.holc_grade.isin(['A','B','C','D']))&
         (df_join3.obesity.notna())
        ][['obesity', 'diabetes', 'bphigh']].describe()

,obesity,diabetes,bphigh
count,6981.000000,6981.000000,6767.000000
mean,32.327876,11.767755,31.345884
std,9.211756,4.759584,9.100300
min,13.700000,1.100000,6.400000
25%,25.000000,8.400000,25.100000
50%,31.500000,11.000000,29.200000
75%,38.400000,14.300000,36.350000
max,59.900000,35.200000,65.400000


In [21]:
# Describe across HOLC grades

grade_counts = df_join3[(df_join3.overlap20_40 == 1)&
                        (df_join3.holc_grade.isin(['A','B','C','D']))&
                        (df_join3.obesity.notna())
        ].groupby('holc_grade')['holc_grade'].value_counts()

grade_percentages = grade_counts / grade_counts.sum() * 100

# Combine counts and percentages in a DataFrame
result = pd.DataFrame({
    'Counts': grade_counts,
    'Percentages': grade_percentages
})

print(grade_counts.sum())
result

6981


,Counts,Percentages
holc_grade,,
A,330,4.727116
B,1337,19.151984
C,3390,48.560378
D,1924,27.560521


In [26]:
# Describe other variables

df_join3[(df_join3.overlap20_40 == 1)&
                        (df_join3.holc_grade.isin(['A','B','C','D']))&
                        (df_join3.obesity.notna())
        ].groupby('holc_grade').describe()

ALAND                                                              
             count          mean           std      min        25%        50%   
holc_grade                                                                      
A            330.0  1.820779e+06  1.362947e+06  51643.0  920338.25  1491565.0  \
B           1337.0  1.012322e+06  8.439583e+05  38660.0  425237.00   844707.0   
C           3390.0  8.628048e+05  8.482750e+05  47372.0  307084.50   616031.5   
D           1924.0  8.400980e+05  1.005848e+06  49643.0  249717.25   515624.0   

                                    AWATER                                     
                   75%         max   count          mean            std  min   
holc_grade                                                                     
A           2337624.50   8744712.0   330.0  88143.148485  343178.598910  0.0  \
B           1314762.00   9206825.0  1337.0  28077.106208  158966.226173  0.0   
C           1153996.75  10011230.0  3390.0  18751.570796  108552.293248  0.0   
D           1045587.50  13034236.0  1924.0  24281.531705  112429.839349  0.0   

                                        tract_area_m2                 
            25%  50%     75%        max         count          mean   
holc_grade                                                            
A           0.0  0.0  6051.5  3533560.0         330.0  1.908922e+06  \
B           0.0  0.0     0.0  2406530.0        1337.0  1.040399e+06   
C           0.0  0.0     0.0  2365806.0        3390.0  8.815563e+05   
D           0.0  0.0     0.0  1980909.0        1924.0  8.643796e+05   

                                                                      
                     std           min            25%           50%   
holc_grade                                                            
A           1.456590e+06  51641.989161  932022.194843  1.541075e+06  \
B           8.936001e+05  38659.846437  425318.066676  8.557859e+05   
C           8.832971e+05  47372.764915  307960.978295  6.196130e+05   
D           1.050462e+06  49641.687330  251120.387219  5.205191e+05   

                                       area_pct                                 
                     75%           max    count      mean       std       min   
holc_grade                                                                      
A           2.374922e+06  8.771742e+06    330.0  0.777391  0.228575  0.250176  \
B           1.344838e+06  9.352341e+06   1337.0  0.840841  0.215467  0.250659   
C           1.171295e+06  1.091060e+07   3390.0  0.837862  0.211941  0.251482   
D           1.081102e+06  1.501515e+07   1924.0  0.825023  0.221136  0.250159   

                                                   index_right                
                 25%       50%       75%       max       count         mean   
holc_grade                                                                    
A           0.620178  0.853332  0.997886  1.000000       330.0  3731.884848  \
B           0.719463  0.966605  1.000000  1.757062      1337.0  4282.628272   
C           0.710702  0.963399  1.000000  1.160249      3390.0  4859.610914   
D           0.677507  0.942276  1.000000  1.010723      1924.0  4745.049896   

                                                                SHAPE_AREA   
                    std   min      25%     50%      75%     max      count   
holc_grade                                                                   
A           2179.227695  17.0  1673.50  3814.0  5654.25  7409.0      330.0  \
B           2364.923241   1.0  2168.00  4627.0  6598.00  7451.0     1337.0   
C           2246.798693   0.0  3068.25  5417.0  6922.50  7453.0     3390.0   
D           2328.015489   2.0  3039.50  5372.0  6749.00  7463.0     1924.0   

                                                                      
                    mean           std            min           25%   
holc_grade                                                            
A     

In [ ]:
# Save the dataset
df_join3[(df_join3.overlap20_40 == 1)&
                        (df_join3.holc_grade.isin(['A','B','C','D']))&
                        (df_join3.obesity.notna())
        ].to_csv(r'Data\dataset.csv')

In [16]:
# Describe the rest of the data not used in the study

df_join3[(df_join3.overlap20_40 == 0)&
         (df_join3.holc_grade.isin(['A','B','C','D']))&
         (df_join3.obesity.notna())
        ][['obesity', 'diabetes', 'bphigh']].describe()

,obesity,diabetes,bphigh
count,7335.000000,7335.000000,6792.000000
mean,33.700791,11.482427,32.950015
std,8.955953,4.611014,8.272719
min,11.500000,1.100000,6.900000
25%,27.000000,8.300000,27.800000
50%,33.400000,10.500000,32.200000
75%,39.700000,13.800000,37.200000
max,63.900000,42.200000,72.300000


In [23]:
# Describe the rest of the data not used in the study

grade_counts = df_join3[(df_join3.overlap20_40 == 0)&
         (df_join3.holc_grade.isin(['A','B','C','D']))&
         (df_join3.overlay_category == 'greater_or_equal_threshold')
        ].groupby('holc_grade')['holc_grade'].value_counts()

grade_percentages = grade_counts / grade_counts.sum() * 100

# Combine counts and percentages in a DataFrame
result = pd.DataFrame({
    'Counts': grade_counts,
    'Percentages': grade_percentages
})

print(grade_counts.sum())
result

4878


,Counts,Percentages
holc_grade,,
A,406,8.323083
B,1072,21.976220
C,2027,41.553916
D,1373,28.146781


In [24]:
# Describe the rest of the data not used in the study

df_join3[(df_join3.overlap20_40 == 0)&
         (df_join3.holc_grade.isin(['A','B','C','D']))&
         (df_join3.overlay_category == 'greater_or_equal_threshold')
        ][['obesity', 'diabetes', 'bphigh']].describe()

,obesity,diabetes,bphigh
count,4314.000000,4314.000000,3912.000000
mean,34.365090,11.687251,33.007975
std,9.375538,4.872042,8.572286
min,11.500000,1.100000,8.300000
25%,27.100000,8.200000,27.600000
50%,34.200000,10.700000,32.100000
75%,40.900000,14.300000,37.625000
max,63.900000,36.300000,67.500000
